<b>Задание 6. Задача об оптимальном использовании ресурсов</b><br>
Для производства трех видов изделий (A, B, C) используются ресурсы типа I и II,<br>
причем закупки ресурсов ограничены возможностями поставщиков. Нормы расхода ресурсов и их запасы приведены в таблице.<br>
1. Постройте математическую модель задачи.<br>
2. Определите такой план производства, при котором стоимость произведенного товара из имеющихся ресурсов является наибольшей.<br>
3. Постройте задачу, двойственную к данной.<br>
4. Найдите решение двойственной задачи. Поясните смысл двойственных переменных.<br>
5. Проверьте результаты вычислений на компьютере.

<table>
    <thead>
        <tr>
            <th></th>
            <th colspan="3">Изделия</th>
            <th></th>
        </tr>
        <tr>
            <th>Ресурсы</th>
            <th>A</th>
            <th>B</th>
            <th>C</th>
            <th>Запасы</th>
        </tr>
    </thead>
    <tr>
        <th>I</th>
        <th>1</th>
        <th>3</th>
        <th>a</th>
        <th>3000</th>
    </tr>
    <tr>
        <th>II</th>
        <th>6</th>
        <th>5</th>
        <th>2</th>
        <th>3320</th>
    </tr>
    <tr>
        <th>Цена</th>
        <th>6b + 12</th>
        <th>5b + 22</th>
        <th>c</th>
        <th></th>
    </tr>
    <tr>
        <th>План</th>
        <th>x1</th>
        <th>x2</th>
        <th>x3</th>
        <th></th>
    </tr>
<table>

<b>Математическая модель задачи.</b><br>
Необходимо максимизировать функцию<br>
*F(x1,x2,x3) = (6b + 12)\*x1 + (5b + 22)\*x2 + c\*x3* → max<br>
при заданных ограничениях ресурсов:<br>
*x1 + 3\*x2 + a\*x3 <= 3000*<br>
*6\*x1 + 5\*x2 + 2\*x3 <= 3320*<br>
*x1, x2, x3 >= 0*

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
import sympy as sym
import matplotlib.pyplot as plt
from gurobipy import *

In [3]:
a = int(input('a = '))
b = int(input('b = '))
c = int(input('c = '))

a = 4
b = 4
c = 32


#### Находим оптимальный план производства

1. используя linprog

In [4]:
#Вводим границы

bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])

#Вводим ограничения

#A_ub - матрица коэф. при x1,x2,x3 в ограничениях, A_ub @ x <= b_ub
#c0 - коэффициенты перед переменными в целевой функции
A_ub = np.array([[1, 3, a], [6, 5, 2]])
b_ub = np.array([3000, 3320])
c0 = np.array([6*b + 12, 5*b + 22, c])

res = linprog(-c0, A_ub, b_ub, bounds=bounds, method='simplex', options={'disp': True})

x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
F_max = -res.fun
print(res.x)

Optimization terminated successfully.
         Current function value: -33360.000000
         Iterations: 2
[  0. 520. 360.]


In [5]:
print(f'Оптимальные значения: x1 = {x1.round(3)}, x2 = {x2.round(3)}, x3 = {x3.round(3)}')
print('Max значение функции:',F_max.round(3))

Оптимальные значения: x1 = 0.0, x2 = 520.0, x3 = 360.0
Max значение функции: 33360.0


In [6]:
b_opt = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3]) # Реальный расход ресурсов
b_opt

array([3000., 3320.])

In [7]:
#Дефицит (исчерпанность ресурсов)
abs(b_ub-b_opt)

array([0., 0.])

2. используя gurobi

In [8]:
var_names = ['x1', 'x2', 'x3']                         
profit = np.array([6*b + 12, 5*b + 22, c])                         # задаем цены
res_used = np.array([[1, 3, a], [6, 5, 2]])  # задаем использование ресурсов
res_avail = np.array([3000, 3320])                    # задаем запасы

n = len(profit)                                    
m = len(res_avail)                                 
model = Model()
x = model.addVars(n, name=var_names) # заводим переменные в модель
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=var_names[i]))
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x92015beb
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [3e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1000000e+32   3.625000e+30   1.100000e+02      0s
       3    3.3360000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.336000000e+04


In [13]:
# выводим параметры модели
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

X = [0.0, 520.0, 360.0] 
Fmax = 33360.0 
Оптимальный план двойственной задачи Pi → (y1,y2) = [5.428571428571429, 5.142857142857142]
x1 = 0.0
Pi1 = 5.428571428571429


In [10]:
# Оценка устойчивости коэффициентов целевой функции
# Минимальные и максимальные значения коэффициентов
# в целевой функции, сохраняющие оптимальность
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1            0           36         -inf      36.2857 
          x2          520           42      41.8182           80 
          x3          360           32         16.8      32.3077 


In [11]:
#Решение двойственной к M задачи
print('Анализ чувствительности\nFmax =', model.ObjVal)
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])

Анализ чувствительности
Fmax = 33360.0

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1            0    -0.285714            0     -387.692      330.909          inf            0          inf 
          x2          520            0            0         -inf          520          inf          520          inf 
          x3          360            0            0         -inf          360          inf          360          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0      5.42857         3000         1992         6640 
          x2            <            0      5.14286         3320         1500         5000 
